# Description

Critical hits work a bit differently in this RPG. If you roll the maximum value on a die, you get to roll the die again and add both dice rolls to get your final score. Critical hits can stack indefinitely -- a second max value means you get a third roll, and so on. With enough luck, any number of points is possible.

## Input

d -- The number of sides on your die.
h -- The amount of health left on the enemy.

## Output

The probability of you getting h or more points with your die.

## Challenge Inputs and Outputs

    Input: d	Input: h	Output
    4	1	1
    4	4	0.25
    4	5	0.25
    4	6	0.1875
    1	10	1
    100	200	0.0001
    8	20	0.009765625

# Secret, off-topic math bonus round

What's the expected (mean) value of a D4? (if you are hoping for as high a total as possible).

In [77]:
def rpg_monster_score_prob(die_size, monster_health): 
    # Easy case: the die only has to be rolled once to kill 
    # the monster with a probability of 
    if die_size >= monster_health:
        return (die_size + 1 - monster_health) / die_size
    
    # If the monster health is higher than the die size,
    # you MUST get the max value to defeat the monster
    # and then we roll again to try to defeat the monster
    return (1. / die_size) * rpg_monster_score_prob(die_size, monster_health - die_size)
    
for stats in [(4, 1), (4, 4), (4, 5), (4, 6), (1, 10), (100, 200), (8, 20)]:
    print('For d={} h={}, prob={}'.format(*stats, round(rpg_monster_score_prob(*stats), 10)))
    

For d=4 h=1, prob=1.0
For d=4 h=4, prob=0.25
For d=4 h=5, prob=0.25
For d=4 h=6, prob=0.1875
For d=1 h=10, prob=1.0
For d=100 h=200, prob=0.0001
For d=8 h=20, prob=0.009765625


## D4 dice expected value

In [79]:
# A D4 usually has a expected value of 3.5
def mean(l):
    return sum(l)/len(l)

mean(range(1,7))

3.5

Since a D4 has an expected value of 3.5:

E(D4) = $\frac{1}{6}\sum_{i=1}^6{d_i} = 3.5$

The new model will shift the expected value to the right...

### Monte Carlo expectation

In [278]:
from random import randint

def dice_rolls(n, sides, mean = True):
    v = 0.
    for _ in range(n):
        val = round(randint(1, sides))
        if val == sides:
            val += dice_rolls(1, sides, mean = False)
        v += val

    return v/n if mean else v

dice_rolls(1000000, 6)
    

4.201136

Using a simple random number generator, the output converges to approximately 4.2. 

Below we can do a simple approximation with only 1 re-roll allowed if the maximum value is hit:

In [276]:
mean([1,2,3,4,5,9.5])

4.083333333333333

### Using infinite series to find the expectation of our model

Since we know that only the last number gets modified, then the expectation will increase. For only one extra reroll, the model becomes

$$\frac{1}{d} \left( \sum_{i=1}^{d-1}{D_i} + D_d + \frac{1}{d} \sum_{i=1}^{d}{D_i} \right)$$

From above, the expectation for one reroll is exactly $4.08\bar{3}$. If we allow the rerolls to be infinite, then the equation above becomes:

$$\frac{1}{d} \left( \sum_{i=1}^{d-1}{D_i} + D_d + \frac{1}{d} \left( \sum_{i=1}^{d-1}{D_i} + D_d + \dots \right) \right)$$

$$=\sum_{n=1}^\infty\left( \sum_{i=1}^{d-1}\left( \frac{D_i}{d^{n}} \right) + \frac{d}{d^{n}} \right)$$

$$=\sum_{n=1}^\infty \frac{1}{d^n}\sum_{i=1}^{d-1}{D_i} + d\sum_{n=1}^\infty{\frac{1}{d^n}}$$

The geometric sum is easy to find $\sum_{n=1}^\infty \frac{1}{d^n} = \frac{1}{d-1}$. For a 6 sided die, the geometric sum becomes $0.2$. The first half of the sum becomes $0.2*(1+2+3+4+5) = 3$ and the second half becomes $6 * 0.2 = 1.2$, using the geometric sum formula. 

The result, 4.2, matches the monte carlo results we obtained above. 